# 如何禁用并行工具调用
:::info OpenAI专属说明
该API目前仅由OpenAI提供支持。
:::
OpenAI 工具调用默认采用并行方式执行工具调用。这意味着如果我们提出类似"东京、纽约和芝加哥的天气如何？"这样的问题，并且我们有一个获取天气的工具，它会并行调用该工具3次。我们可以通过使用 `parallel_tool_call` 参数强制其仅调用单个工具一次。

首先让我们设置工具和模型：

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [ ]:
import os
from getpass import getpass

from langchain_openai import ChatOpenAI

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

现在让我们快速展示一个禁用并行工具调用的示例：

In [ ]:
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)
llm_with_tools.invoke("Please call the first tool two times").tool_calls

[{'name': 'add',
  'args': {'a': 2, 'b': 2},
  'id': 'call_Hh4JOTCDM85Sm9Pr84VKrWu5'}]

正如我们所见，尽管我们明确指示模型调用工具两次，但通过禁用并行工具调用功能，模型被限制为仅能调用一次。